In [1]:
import os
import autogen
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from autogen import register_function
from typing import Annotated, Literal

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
# def parse_files_in_directory(directory_path):
    
#     parsed_data = parse_pdfs(directory_path)

#     # for filename in os.listdir(directory_path):
#         # file_path = os.path.join(directory_path, filename)
        
#         # if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
#         #     loader = UnstructuredImageLoader(file_path)
#         #     documents = loader.load()
#         #     parsed_text = ' '.join([doc.page_content for doc in documents])

#         # else:
#         #     continue

#         # parsed_data[filename] = parsed_text

#     return parsed_data


def parse_pdfs(directory:str ="../data/") -> dict:
# def parse_pdfs(dir_path="../data/"):
    parsed_data = {}
    loader = PyPDFDirectoryLoader(directory, extract_images=True)
    documents = loader.load()
    for page in documents:
        if parsed_data.get(page.metadata["source"]) is None:
            parsed_data[page.metadata["source"]] = [page.page_content]
        else:
            parsed_data[page.metadata["source"]].append(page.page_content)
    return parsed_data


In [3]:
api_key = os.environ.get("GROQ_API_KEY")

config_list = [
  {
    "model": "llama-3.2-90b-text-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
    "api_rate_limit": 100
  }
]

# llm_config = config_list[0]
llm_config={"config_list": config_list}

In [4]:
# user_proxy = autogen.ConversableAgent(
#     name="Admin",
#     system_message="Give the task, and send instructions to doc_classifier to classify the documents in the data folder",
#     code_execution_config=False,
#     llm_config=llm_config,
#     human_input_mode="ALWAYS",
# )

user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=10,
    # code_execution_config={
    #     "work_dir": "code",
    #     "use_docker": False
    # },
    llm_config=False,
)

In [5]:
doc_parse_caller = autogen.ConversableAgent(
    name="Doc Parse Caller",
    llm_config=llm_config,
    description="A caller for parsing documents",
    system_message="You are a helpful AI assistant. Use the provided python function parse_pdfs(directory), to parse the documents at the directory '../data/'.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

doc_parser = autogen.ConversableAgent(
    name="Doc Parser",
    llm_config=llm_config,
    system_message="You are an executer that parses documents. If you receive a python call instruction, then you execute it, or else ignore the message."
                   "Reply TERMINATE afer you execute the python function",
)

prompt = """
For each document provided, analyze its content and categorize it into one of the following categories: ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other']. Your response should strictly follow this dictionary structured format:

{
  "category_name": "<full_parsed_document_string>",
}

Always include each key exactly as shown. The category_name field should contain the best matching category from the predefined options, and the full_parsed_document_string value should include the full parsed document text. Adhere precisely to this format, without adding or omitting any fields.
"""

doc_classifier = autogen.ConversableAgent(
    name="Doc Classifier",
    llm_config=llm_config,
    description="A classifier that classifies the text that has been parsed",
    system_message=prompt
)




# doc_classifier = autogen.ConversableAgent(
#     name="Doc Classifier",
#     llm_config=llm_config,
#     description="A classifier that classifies the text that has been parsed",
#     system_message="For each document provided, categorize them from one of these options ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other'] "
#                    "and reply with a dictionary in this format {category_name: full_parsed_document}, where full_parsed_document is the whole document that was parsed"
# )

chief_verifier = autogen.ConversableAgent(
    name="Chief Verifier",
    llm_config=llm_config,
    description="The Chief verifier",
    system_message="You are the chief verifier."
                   "You collect the information from the Doc Classifier and pass the information to your team."
)

health_insurance_verifier = autogen.ConversableAgent(
    name="Health Insurance Verifier",
    llm_config=llm_config,
    description="A verifier that checks if health insurance is still valid",
    system_message="You are a helpful AI assistant."
                   "If there exists a Health Insurance key in the dictionary received, then check its corresponding parsed document to see if the validity of the insurance is beyond today."
                #    "Reply TERMINATE when the task is done.",
)

rp_prompt = '''
You are a helpful AI assistant. Your task is to check the contents of a dictionary that contains various document types as keys and their content as values, formatted as follows:

{
  "Passport": "<content>",
  "Health Insurance": "<content>",
  ...
}

If a key named "Residence Permit" is present in the dictionary, examine its content to verify whether the health insurance document is valid beyond today’s date. Provide a precise response indicating whether the residence permit is valid or expired, based on the expiration information in the document.
'''

rp_verifier = autogen.ConversableAgent(
    name="Residence Permit Verifier",
    llm_config=llm_config,
    description="A verifier that checks if residence permit is still valid",
    system_message=rp_prompt
)

passport_verifier = autogen.ConversableAgent(
    name="Passport Verifier",
    llm_config=llm_config,
    description="A verifier that checks passport validity",
    system_message="You are a helpful AI assistant."
                   "If there exists a passport document, then check if the expiry date is more than a year from today"
                #    "Reply TERMINATE when the task is done.",
)



In [6]:
register_function(
    parse_pdfs,
    caller=doc_parse_caller,  
    executor=doc_parser,  
    name="parse_pdfs",  
    description="A simple document parser",  # A description of the tool.
)

# # Register the tool signature with the assistant agent.
# doc_parse_caller.register_for_llm(name="parse_pdfs", description="A simple pdf parser")(parse_pdfs)

# # Register the tool function with the user proxy agent.
# doc_parser.register_for_execution(name="parse_pdfs")(parse_pdfs)

In [7]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, doc_parse_caller, doc_parser, doc_classifier, chief_verifier, health_insurance_verifier, passport_verifier, rp_verifier],
    messages=[],
    max_round=5,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [doc_parse_caller],
        doc_parse_caller: [doc_parser],
        doc_parser: [doc_classifier],
        doc_classifier: [chief_verifier],
        chief_verifier: [health_insurance_verifier, rp_verifier],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
doc_parse_caller.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple document parser',
   'name': 'parse_pdfs',
   'parameters': {'type': 'object',
    'properties': {'directory': {'type': 'string',
      'default': '../data/',
      'description': 'directory'}},
    'required': []}}}]

In [9]:
# def parse_calling(recipient, messages, sender, config):
#     # return f"Can you provide the python function to call for parsing the documents?\n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"
#     #print(f"{recipient.chat_messages_for_summary(sender)[-1]['content']}")
    
#     # if isinstance(recipient.chat_messages_for_summary(sender)[-1]['content'], dict):
#     #     return "TERMINATE"
#     # else:
#     return "I have some pdf documents in the directory '../data/', and I would like to parse the documents in this directory"


# def parsed_docs(recipient, messages, sender, config):
#     # return f"Can you provide the python function to call for parsing the documents?\n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"
#     #print(f"{recipient.chat_messages_for_summary(sender)[-1]['content']}")
#     return "Here are the parsed documents to be classified"


chief_verifier.register_nested_chats(
    [
        {
            "recipient": rp_verifier,
            # "message": parse_calling,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : 
                "Return review into as JSON object only:"
                "{'Verifier': '', 'Verification': ''}. Here Verifier should be your role",},
        },
        {
            "recipient": passport_verifier,
            # "message": parse_calling,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : 
                "Return review into as JSON object only:"
                "{'Verifier': '', 'Verification': ''}. Here Verifier should be your role",},
        },
    ],
    trigger=manager,  # condition=my_condition,
)

In [10]:
result = user_proxy.initiate_chat(
    manager,
    # message="Parse the documents and classify each of them by a heading",
    message="I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)",
)

User (to chat_manager):

I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)

--------------------------------------------------------------------------------

Next speaker: Doc Parse Caller


>>>>>>>> USING AUTO REPLY...
Doc Parse Caller (to chat_manager):

***** Suggested tool call (call_w2sj): parse_pdfs *****
Arguments: 
{"directory": "../data/"}
*******************************************************

--------------------------------------------------------------------------------

Next speaker: Doc Parser


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION parse_pdfs...


c:\Users\ahmad\Desktop\LlamaHackers\venv\Lib\site-packages\autogen\oai\groq.py:280: UserWarning: Cost calculation not available for model llama-3.2-90b-text-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Doc Parser (to chat_manager):

Doc Parser (to chat_manager):

***** Response from calling tool (call_w2sj) *****
{"..\\data\\belgian_doc.pdf": ["BEL 2 TITRE DE SEJOUR 5261785 20 5261785 20 NOM Prénoms/SURNAME Farenames AHMAD \nNATIONAEFE NATIONAL M/M BGD \nSEXE DATE DE NAISSANCE SEX DATE OF BIRTH 05 12 1996 CAT. DU TITRE TYPE OF PERMIT VALABLE JUSQU AU/CARD EXPIRY A. SEJOUR LIMITE 31 10 2024 \nREMARQUES REMARKS N REGISTRE NATIONAL 96.12.05-751.35 \nwwww.erucoueei \nRESIDENCE PERMIT \nREMARQUES / REMARKS 33A0 MARCHE DU TRAVAIL IMITE ETUDIANT, PROGRAMME DE MOBILITTE DATE DE DELIVRANCE LIEU DE DELIVRANCE PLACE OF ISSUE DATE OF IssUE BRUXELLES 19 01 2023 LIEU DE NAISSANCE/PLACE OF BIRTH KUALA LUMPUR \nEUEU \nIDBEL5261785204<<<<<<<<<<<<<<< 9612051 M2410317BGD961205751352 AHMAD<<<<<<<<<<<<<<<<<<<<<<<<< 526178520\nDATE DE NAISSANCE/\nDATE OF BIRTH\n05121996\nVALABLE JUSQU'AU / CARD EXPIRY\n31102024\nN8REGISTRE NATI0NAL 96.12.05-751.35\nCAT.DU TITRE/ TYPE OF PERMIT\nA.SEJOUR LIMITE\nRESIDENCE 